# 模型训练和评估验证
* 数据来源：../datasets/data_feature_processed.csv
* 代码内容：训练xgboost模型，并进行线下评估

# 导入包

In [5]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import time
import re
%matplotlib inline

# 导入特征工程后的数据

In [6]:
inputfile = '../datasets/data_feature_processed.csv'
modelfile = '../model/xgb.pkl'

In [7]:
data = pd.read_csv(inputfile)

# XGBoost模型训练

In [9]:
df = data.copy()
df.sort_values(by=['loc_id','time_stamp'],inplace=True)

## 1. 划分训练集和测试集

In [42]:
train = df[df.time_stamp<'2017-10-01']
test = df[df.time_stamp>='2017-10-01']
train_X = train.iloc[:,3:]
train_y = train['num_of_people']
test_X = test.iloc[:,3:]
test_y = test['num_of_people']
print(train_X.shape,test_X.shape)

(191664, 90) (18216, 90)


## 2. XGBoost训练

In [43]:
model_xgb = xgboost_model_train(train_X,train_y,test_X,test_y)

[0]	train-rmse:594.044	test-rmse:1038.57
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 20 rounds.
[1]	train-rmse:577.615	test-rmse:1011.55
[2]	train-rmse:561.494	test-rmse:985.231
[3]	train-rmse:545.966	test-rmse:959.46
[4]	train-rmse:530.817	test-rmse:934.149
[5]	train-rmse:516.123	test-rmse:909.949
[6]	train-rmse:501.948	test-rmse:886.75
[7]	train-rmse:488.159	test-rmse:863.722
[8]	train-rmse:474.848	test-rmse:841.794
[9]	train-rmse:462.169	test-rmse:821.043
[10]	train-rmse:449.603	test-rmse:800.743
[11]	train-rmse:437.695	test-rmse:781.336
[12]	train-rmse:425.886	test-rmse:761.875
[13]	train-rmse:414.689	test-rmse:743.05
[14]	train-rmse:403.776	test-rmse:724.929
[15]	train-rmse:393.045	test-rmse:707.286
[16]	train-rmse:382.646	test-rmse:689.634
[17]	train-rmse:372.777	test-rmse:672.284
[18]	train-rmse:363.172	test-rmse:656.029
[19]	train-rmse:353.794	test-rmse:640.827
[20]	train-rmse:344.621	test-r

[191]	train-rmse:87.4698	test-rmse:203.643
[192]	train-rmse:87.3849	test-rmse:203.644
[193]	train-rmse:87.311	test-rmse:203.596
[194]	train-rmse:87.2734	test-rmse:203.544
[195]	train-rmse:87.1741	test-rmse:203.454
[196]	train-rmse:87.0995	test-rmse:203.43
[197]	train-rmse:87.002	test-rmse:203.315
[198]	train-rmse:86.9296	test-rmse:203.278
[199]	train-rmse:86.8816	test-rmse:203.252
[200]	train-rmse:86.8186	test-rmse:203.228
[201]	train-rmse:86.759	test-rmse:203.271
[202]	train-rmse:86.681	test-rmse:203.233
[203]	train-rmse:86.5949	test-rmse:203.123
[204]	train-rmse:86.5004	test-rmse:203.098
[205]	train-rmse:86.4144	test-rmse:203.084
[206]	train-rmse:86.3432	test-rmse:202.955
[207]	train-rmse:86.26	test-rmse:202.898
[208]	train-rmse:86.206	test-rmse:202.89
[209]	train-rmse:86.1771	test-rmse:202.924
[210]	train-rmse:86.1101	test-rmse:202.857
[211]	train-rmse:86.0479	test-rmse:202.846
[212]	train-rmse:85.9822	test-rmse:202.78
[213]	train-rmse:85.9427	test-rmse:202.791
[214]	train-rmse:85.8

[383]	train-rmse:78.2701	test-rmse:200.277
[384]	train-rmse:78.2332	test-rmse:200.278
[385]	train-rmse:78.2002	test-rmse:200.302
[386]	train-rmse:78.1361	test-rmse:200.288
[387]	train-rmse:78.0941	test-rmse:200.286
[388]	train-rmse:78.0873	test-rmse:200.284
[389]	train-rmse:78.0125	test-rmse:200.286
[390]	train-rmse:77.9744	test-rmse:200.258
[391]	train-rmse:77.9134	test-rmse:200.242
[392]	train-rmse:77.8919	test-rmse:200.246
[393]	train-rmse:77.8336	test-rmse:200.204
[394]	train-rmse:77.7555	test-rmse:200.114
[395]	train-rmse:77.7423	test-rmse:200.132
[396]	train-rmse:77.665	test-rmse:200.077
[397]	train-rmse:77.6374	test-rmse:200.066
[398]	train-rmse:77.6062	test-rmse:200.055
[399]	train-rmse:77.5793	test-rmse:200.058
[400]	train-rmse:77.5286	test-rmse:200.062
[401]	train-rmse:77.5062	test-rmse:200.084
[402]	train-rmse:77.4604	test-rmse:200.069
[403]	train-rmse:77.4283	test-rmse:200.073
[404]	train-rmse:77.4144	test-rmse:200.088
[405]	train-rmse:77.4088	test-rmse:200.109
[406]	train-

## 3. 模型评估
* 使用10月份数据进行实时预测

In [47]:
Oct_test_df = pd.read_csv('../datasets/Oct_test_data.csv',parse_dates=['time_stamp'])

In [48]:
%time pred_df = real_time_predict(model_xgb,Oct_test_df)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


[180.]
第0行预测完成！
[129.]
第1行预测完成！
[110.]
第2行预测完成！
[109.]
第3行预测完成！
[116.]
第4行预测完成！
[116.]
第5行预测完成！
[167.]
第6行预测完成！
[401.]
第7行预测完成！
[617.]
第8行预测完成！
[662.]
第9行预测完成！
[511.]
第10行预测完成！
[775.]
第11行预测完成！
[1025.]
第12行预测完成！
[807.]
第13行预测完成！
[572.]
第14行预测完成！
[510.]
第15行预测完成！
[511.]
第16行预测完成！
[805.]
第17行预测完成！
[814.]
第18行预测完成！
[562.]
第19行预测完成！
[526.]
第20行预测完成！
[535.]
第21行预测完成！
[562.]
第22行预测完成！
[351.]
第23行预测完成！
[188.]
第24行预测完成！
[143.]
第25行预测完成！
[129.]
第26行预测完成！
[139.]
第27行预测完成！
[146.]
第28行预测完成！
[141.]
第29行预测完成！
[180.]
第30行预测完成！
[498.]
第31行预测完成！
[737.]
第32行预测完成！
[683.]
第33行预测完成！
[527.]
第34行预测完成！
[820.]
第35行预测完成！
[1020.]
第36行预测完成！
[775.]
第37行预测完成！
[575.]
第38行预测完成！
[525.]
第39行预测完成！
[503.]
第40行预测完成！
[841.]
第41行预测完成！
[806.]
第42行预测完成！
[567.]
第43行预测完成！
[526.]
第44行预测完成！
[539.]
第45行预测完成！
[541.]
第46行预测完成！
[343.]
第47行预测完成！
[182.]
第48行预测完成！
[139.]
第49行预测完成！
[119.]
第50行预测完成！
[119.]
第51行预测完成！
[121.]
第52行预测完成！
[118.]
第53行预测完成！
[191.]
第54行预测完成！
[546.]
第55行预测完成！
[748.]
第56行预测完成！
[656.]
第57行预测完成！
[503.]
第58行预测完成！
[823.

[825.]
第467行预测完成！
[935.]
第468行预测完成！
[687.]
第469行预测完成！
[565.]
第470行预测完成！
[529.]
第471行预测完成！
[514.]
第472行预测完成！
[780.]
第473行预测完成！
[784.]
第474行预测完成！
[538.]
第475行预测完成！
[476.]
第476行预测完成！
[449.]
第477行预测完成！
[441.]
第478行预测完成！
[257.]
第479行预测完成！
[157.]
第480行预测完成！
[120.]
第481行预测完成！
[108.]
第482行预测完成！
[99.]
第483行预测完成！
[97.]
第484行预测完成！
[104.]
第485行预测完成！
[176.]
第486行预测完成！
[319.]
第487行预测完成！
[470.]
第488行预测完成！
[570.]
第489行预测完成！
[512.]
第490行预测完成！
[779.]
第491行预测完成！
[867.]
第492行预测完成！
[625.]
第493行预测完成！
[531.]
第494行预测完成！
[507.]
第495行预测完成！
[518.]
第496行预测完成！
[738.]
第497行预测完成！
[762.]
第498行预测完成！
[525.]
第499行预测完成！
[472.]
第500行预测完成！
[460.]
第501行预测完成！
[454.]
第502行预测完成！
[268.]
第503行预测完成！
[171.]
第504行预测完成！
[141.]
第505行预测完成！
[127.]
第506行预测完成！
[122.]
第507行预测完成！
[116.]
第508行预测完成！
[121.]
第509行预测完成！
[196.]
第510行预测完成！
[528.]
第511行预测完成！
[784.]
第512行预测完成！
[707.]
第513行预测完成！
[558.]
第514行预测完成！
[945.]
第515行预测完成！
[1100.]
第516行预测完成！
[853.]
第517行预测完成！
[680.]
第518行预测完成！
[602.]
第519行预测完成！
[587.]
第520行预测完成！
[1035.]
第521行预测完成！
[974.]
第52

第380行预测完成！
[601.]
第381行预测完成！
[617.]
第382行预测完成！
[359.]
第383行预测完成！
[203.]
第384行预测完成！
[198.]
第385行预测完成！
[210.]
第386行预测完成！
[208.]
第387行预测完成！
[209.]
第388行预测完成！
[209.]
第389行预测完成！
[267.]
第390行预测完成！
[626.]
第391行预测完成！
[759.]
第392行预测完成！
[634.]
第393行预测完成！
[501.]
第394行预测完成！
[883.]
第395行预测完成！
[1158.]
第396行预测完成！
[840.]
第397行预测完成！
[650.]
第398行预测完成！
[589.]
第399行预测完成！
[546.]
第400行预测完成！
[1011.]
第401行预测完成！
[956.]
第402行预测完成！
[641.]
第403行预测完成！
[581.]
第404行预测完成！
[603.]
第405行预测完成！
[616.]
第406行预测完成！
[367.]
第407行预测完成！
[200.]
第408行预测完成！
[200.]
第409行预测完成！
[211.]
第410行预测完成！
[207.]
第411行预测完成！
[205.]
第412行预测完成！
[206.]
第413行预测完成！
[270.]
第414行预测完成！
[613.]
第415行预测完成！
[766.]
第416行预测完成！
[615.]
第417行预测完成！
[492.]
第418行预测完成！
[871.]
第419行预测完成！
[1186.]
第420行预测完成！
[846.]
第421行预测完成！
[649.]
第422行预测完成！
[592.]
第423行预测完成！
[565.]
第424行预测完成！
[998.]
第425行预测完成！
[965.]
第426行预测完成！
[643.]
第427行预测完成！
[571.]
第428行预测完成！
[600.]
第429行预测完成！
[608.]
第430行预测完成！
[360.]
第431行预测完成！
[191.]
第432行预测完成！
[186.]
第433行预测完成！
[182.]
第434行预测完成！
[183.]
第435行预测

第280行预测完成！
[1762.]
第281行预测完成！
[1830.]
第282行预测完成！
[1258.]
第283行预测完成！
[1043.]
第284行预测完成！
[1065.]
第285行预测完成！
[961.]
第286行预测完成！
[384.]
第287行预测完成！
[120.]
第288行预测完成！
[88.]
第289行预测完成！
[80.]
第290行预测完成！
[86.]
第291行预测完成！
[97.]
第292行预测完成！
[145.]
第293行预测完成！
[321.]
第294行预测完成！
[937.]
第295行预测完成！
[1271.]
第296行预测完成！
[1057.]
第297行预测完成！
[956.]
第298行预测完成！
[1554.]
第299行预测完成！
[2014.]
第300行预测完成！
[1389.]
第301行预测完成！
[1012.]
第302行预测完成！
[1042.]
第303行预测完成！
[985.]
第304行预测完成！
[1569.]
第305行预测完成！
[1570.]
第306行预测完成！
[1126.]
第307行预测完成！
[967.]
第308行预测完成！
[923.]
第309行预测完成！
[844.]
第310行预测完成！
[359.]
第311行预测完成！
[146.]
第312行预测完成！
[113.]
第313行预测完成！
[111.]
第314行预测完成！
[114.]
第315行预测完成！
[124.]
第316行预测完成！
[155.]
第317行预测完成！
[298.]
第318行预测完成！
[660.]
第319行预测完成！
[945.]
第320行预测完成！
[900.]
第321行预测完成！
[879.]
第322行预测完成！
[1392.]
第323行预测完成！
[1718.]
第324行预测完成！
[1188.]
第325行预测完成！
[940.]
第326行预测完成！
[871.]
第327行预测完成！
[895.]
第328行预测完成！
[1365.]
第329行预测完成！
[1403.]
第330行预测完成！
[1021.]
第331行预测完成！
[872.]
第332行预测完成！
[807.]
第333行预测完成！
[782.]
第334行预测完成！


...

第246行预测完成！
[761.]
第247行预测完成！
[999.]
第248行预测完成！
[868.]
第249行预测完成！
[711.]
第250行预测完成！
[1066.]
第251行预测完成！
[1132.]
第252行预测完成！
[896.]
第253行预测完成！
[737.]
第254行预测完成！
[637.]
第255行预测完成！
[664.]
第256行预测完成！
[990.]
第257行预测完成！
[981.]
第258行预测完成！
[695.]
第259行预测完成！
[635.]
第260行预测完成！
[688.]
第261行预测完成！
[743.]
第262行预测完成！
[538.]
第263行预测完成！
[451.]
第264行预测完成！
[397.]
第265行预测完成！
[363.]
第266行预测完成！
[357.]
第267行预测完成！
[352.]
第268行预测完成！
[352.]
第269行预测完成！
[437.]
第270行预测完成！
[732.]
第271行预测完成！
[974.]
第272行预测完成！
[829.]
第273行预测完成！
[685.]
第274行预测完成！
[1003.]
第275行预测完成！
[1077.]
第276行预测完成！
[859.]
第277行预测完成！
[728.]
第278行预测完成！
[642.]
第279行预测完成！
[646.]
第280行预测完成！
[917.]
第281行预测完成！
[881.]
第282行预测完成！
[621.]
第283行预测完成！
[575.]
第284行预测完成！
[600.]
第285行预测完成！
[705.]
第286行预测完成！
[496.]
第287行预测完成！
[403.]
第288行预测完成！
[387.]
第289行预测完成！
[387.]
第290行预测完成！
[395.]
第291行预测完成！
[374.]
第292行预测完成！
[375.]
第293行预测完成！
[455.]
第294行预测完成！
[654.]
第295行预测完成！
[867.]
第296行预测完成！
[780.]
第297行预测完成！
[661.]
第298行预测完成！
[891.]
第299行预测完成！
[958.]
第300行预测完成！
[802.]
第301行预

第149行预测完成！
[162.]
第150行预测完成！
[497.]
第151行预测完成！
[778.]
第152行预测完成！
[871.]
第153行预测完成！
[847.]
第154行预测完成！
[1394.]
第155行预测完成！
[1653.]
第156行预测完成！
[1186.]
第157行预测完成！
[900.]
第158行预测完成！
[811.]
第159行预测完成！
[866.]
第160行预测完成！
[1313.]
第161行预测完成！
[1488.]
第162行预测完成！
[1107.]
第163行预测完成！
[903.]
第164行预测完成！
[859.]
第165行预测完成！
[840.]
第166行预测完成！
[356.]
第167行预测完成！
[77.]
第168行预测完成！
[41.]
第169行预测完成！
[41.]
第170行预测完成！
[39.]
第171行预测完成！
[39.]
第172行预测完成！
[54.]
第173行预测完成！
[170.]
第174行预测完成！
[803.]
第175行预测完成！
[1188.]
第176行预测完成！
[1118.]
第177行预测完成！
[990.]
第178行预测完成！
[1805.]
第179行预测完成！
[2277.]
第180行预测完成！
[1558.]
第181行预测完成！
[1162.]
第182行预测完成！
[1144.]
第183行预测完成！
[1180.]
第184行预测完成！
[1853.]
第185行预测完成！
[1871.]
第186行预测完成！
[1326.]
第187行预测完成！
[1146.]
第188行预测完成！
[1160.]
第189行预测完成！
[1159.]
第190行预测完成！
[454.]
第191行预测完成！
[83.]
第192行预测完成！
[45.]
第193行预测完成！
[33.]
第194行预测完成！
[27.]
第195行预测完成！
[26.]
第196行预测完成！
[56.]
第197行预测完成！
[200.]
第198行预测完成！
[1142.]
第199行预测完成！
[1425.]
第200行预测完成！
[1199.]
第201行预测完成！
[968.]
第202行预测完成！
[1766.]
第203行预测完成！
[2379

第56行预测完成！
[490.]
第57行预测完成！
[414.]
第58行预测完成！
[684.]
第59行预测完成！
[949.]
第60行预测完成！
[654.]
第61行预测完成！
[538.]
第62行预测完成！
[478.]
第63行预测完成！
[443.]
第64行预测完成！
[757.]
第65行预测完成！
[784.]
第66行预测完成！
[523.]
第67行预测完成！
[420.]
第68行预测完成！
[442.]
第69行预测完成！
[456.]
第70行预测完成！
[235.]
第71行预测完成！
[120.]
第72行预测完成！
[96.]
第73行预测完成！
[86.]
第74行预测完成！
[84.]
第75行预测完成！
[86.]
第76行预测完成！
[95.]
第77行预测完成！
[145.]
第78行预测完成！
[463.]
第79行预测完成！
[610.]
第80行预测完成！
[499.]
第81行预测完成！
[393.]
第82行预测完成！
[674.]
第83行预测完成！
[964.]
第84行预测完成！
[638.]
第85行预测完成！
[508.]
第86行预测完成！
[491.]
第87行预测完成！
[439.]
第88行预测完成！
[777.]
第89行预测完成！
[808.]
第90行预测完成！
[520.]
第91行预测完成！
[439.]
第92行预测完成！
[460.]
第93行预测完成！
[474.]
第94行预测完成！
[249.]
第95行预测完成！
[133.]
第96行预测完成！
[111.]
第97行预测完成！
[110.]
第98行预测完成！
[104.]
第99行预测完成！
[102.]
第100行预测完成！
[108.]
第101行预测完成！
[157.]
第102行预测完成！
[458.]
第103行预测完成！
[571.]
第104行预测完成！
[488.]
第105行预测完成！
[378.]
第106行预测完成！
[622.]
第107行预测完成！
[881.]
第108行预测完成！
[612.]
第109行预测完成！
[483.]
第110行预测完成！
[466.]
第111行预测完成！
[421.]
第112行预测完成！
[734.]
第113行预测完成！
[741.]
第114行

[881.]
第522行预测完成！
[587.]
第523行预测完成！
[493.]
第524行预测完成！
[515.]
第525行预测完成！
[535.]
第526行预测完成！
[289.]
第527行预测完成！
[128.]
第528行预测完成！
[105.]
第529行预测完成！
[98.]
第530行预测完成！
[96.]
第531行预测完成！
[90.]
第532行预测完成！
[95.]
第533行预测完成！
[140.]
第534行预测完成！
[485.]
第535行预测完成！
[645.]
第536行预测完成！
[518.]
第537行预测完成！
[383.]
第538行预测完成！
[751.]
第539行预测完成！
[1010.]
第540行预测完成！
[696.]
第541行预测完成！
[552.]
第542行预测完成！
[471.]
第543行预测完成！
[453.]
第544行预测完成！
[887.]
第545行预测完成！
[871.]
第546行预测完成！
[594.]
第547行预测完成！
[499.]
第548行预测完成！
[515.]
第549行预测完成！
[521.]
第550行预测完成！
[284.]
第551行预测完成！
******删减前地点数据大小：1272******
******删减后地点数据大小：552******
[18.]
第0行预测完成！
[19.]
第1行预测完成！
[20.]
第2行预测完成！
[22.]
第3行预测完成！
[25.]
第4行预测完成！
[32.]
第5行预测完成！
[65.]
第6行预测完成！
[231.]
第7行预测完成！
[429.]
第8行预测完成！
[552.]
第9行预测完成！
[504.]
第10行预测完成！
[579.]
第11行预测完成！
[569.]
第12行预测完成！
[545.]
第13行预测完成！
[480.]
第14行预测完成！
[559.]
第15行预测完成！
[587.]
第16行预测完成！
[825.]
第17行预测完成！
[666.]
第18行预测完成！
[466.]
第19行预测完成！
[476.]
第20行预测完成！
[412.]
第21行预测完成！
[313.]
第22行预测完成！
[111.]
第23行预测完成！
[32.]
第24行预测完成！
[17

第439行预测完成！
[774.]
第440行预测完成！
[785.]
第441行预测完成！
[641.]
第442行预测完成！
[919.]
第443行预测完成！
[797.]
第444行预测完成！
[650.]
第445行预测完成！
[611.]
第446行预测完成！
[675.]
第447行预测完成！
[721.]
第448行预测完成！
[944.]
第449行预测完成！
[743.]
第450行预测完成！
[500.]
第451行预测完成！
[460.]
第452行预测完成！
[423.]
第453行预测完成！
[314.]
第454行预测完成！
[99.]
第455行预测完成！
[24.]
第456行预测完成！
[9.]
第457行预测完成！
[5.]
第458行预测完成！
[3.]
第459行预测完成！
[3.]
第460行预测完成！
[11.]
第461行预测完成！
[50.]
第462行预测完成！
[239.]
第463行预测完成！
[446.]
第464行预测完成！
[631.]
第465行预测完成！
[574.]
第466行预测完成！
[744.]
第467行预测完成！
[734.]
第468行预测完成！
[541.]
第469行预测完成！
[503.]
第470行预测完成！
[532.]
第471行预测完成！
[573.]
第472行预测完成！
[758.]
第473行预测完成！
[657.]
第474行预测完成！
[460.]
第475行预测完成！
[398.]
第476行预测完成！
[357.]
第477行预测完成！
[286.]
第478行预测完成！
[78.]
第479行预测完成！
[16.]
第480行预测完成！
[7.]
第481行预测完成！
[3.]
第482行预测完成！
[2.]
第483行预测完成！
[1.]
第484行预测完成！
[6.]
第485行预测完成！
[36.]
第486行预测完成！
[142.]
第487行预测完成！
[290.]
第488行预测完成！
[399.]
第489行预测完成！
[452.]
第490行预测完成！
[581.]
第491行预测完成！
[628.]
第492行预测完成！
[428.]
第493行预测完成！
[373.]
第494行预测完成！
[373.]
第495行预测完成！
[424.]

In [49]:
pred = pred_df.num_of_people
real = test.num_of_people
rmse = np.sqrt(mean_squared_error(pred,real))
print('Test RMSE:',rmse)

Test RMSE: 323.7340279454762


## 实验记录
1. 0填充 训练数据从2月份开始 rmse：358
2. 前一个非null值填充 去掉二月数据 异常值处理 rmse：377
3. 前一个非null值填充 去掉二月数据 rmse：412
4. 0填充 训练数据从3月份开始 rmse：378
5. 按地点前一时刻非空值填充 训练数据从4月份开始 rmse：576
6. 0填充 训练数据从4月份开始 去掉十一 rmse：582
7. 0填充 训练数据从2月份开始 去掉十一 增加小时特征 rmse：323

# 公共函数

In [12]:
# 按地点分组，加快处理速度
def locID_split(data):
    res = []
    df = data.copy()
    for i in df.loc_id.unique():
        tmp = df[df.loc_id == i]
        tmp.reset_index(inplace=True) # ！！！注意重置索引 实时预测使用的loc方法，行索引重置后才能正确填充
        tmp.drop('index',inplace=True,axis=1)
        res.append(tmp)
    return res

In [13]:
# 实时预测填充
# 思路：第一行特征完整，进行预测，预测出来的价格添加到后边time_stamp对应的历史特征上去，接着预测第二行，进行迭代
def real_time_predict(model,data):
    df = data.copy()
    test_list = locID_split(df)
    res = pd.DataFrame()
    for tmp in test_list:
        for i in range(tmp.shape[0]):
            X = xgb.DMatrix(tmp.iloc[i:i+1,3:])
            pred = np.round(model.predict(X,ntree_limit = model.best_iteration))
            print(pred)
            tmp.loc[i,'num_of_people'] = pred
            for j in range(len(columns_24h)):
                tmp.loc[i+j+1,columns_24h[j]] = pred
            for k in range(len(columns_30d)):
                tmp.loc[i+(k+1)*24,columns_30d[k]] = pred
            print('第%s行预测完成！'%i)
        print('******删减前地点数据大小：%s******'%tmp.shape[0])
        tmp.dropna(axis=0,inplace=True) # 数据在填充历史特征时，会多填充不需要的数据，去掉
        print('******删减后地点数据大小：%s******'%tmp.shape[0])
        res = res.append(tmp) # df.append返回DataFrame，注意赋值
    return res

In [14]:
reg = re.compile('\s+|\t|\n') # 正则匹配空格、Tab键、换行
#列名，历史24小时人数
col_24h='''history_1h_population,history_2h_population,history_3h_population,history_4h_population
    ,history_5h_population,history_6h_population,history_7h_population,history_8h_population
    ,history_9h_population,history_10h_population,history_11h_population,history_12h_population
    ,history_13h_population,history_14h_population,history_15h_population,history_16h_population,history_17h_population
    ,history_18h_population,history_19h_population,history_20h_population,history_21h_population,history_22h_population
    ,history_23h_population'''
columns_24h=re.sub(reg,'',col_24h).split(',')  
#列名，同一时刻历史15天人数
col_30d='''history_1d_population,history_2d_population,history_3d_population,
    history_4d_population,history_5d_population,history_6d_population,
    history_7d_population,history_8d_population,history_9d_population,history_10d_population,history_11d_population,
    history_12d_population,history_13d_population,history_14d_population,history_15d_population,history_16d_population
    ,history_17d_population,history_18d_population,history_19d_population,history_20d_population,history_21d_population
    ,history_22d_population,history_23d_population,history_24d_population,history_25d_population,history_26d_population
    ,history_27d_population,history_28d_population,history_29d_population,history_30d_population'''
columns_30d=re.sub(reg,'',col_30d).split(',')  

In [15]:
def xgboost_model_train(train_X,train_y,test_X,test_y):
    # 模型训练
    xgb_train  = xgb.DMatrix(train_X, train_y)
    xgb_test   = xgb.DMatrix(test_X, test_y)
    params_xgb = {
#         'booster': 'gbtree',                # 取决于使用哪种booster,有两中模型可以选择gbtree和gblinear。gbtree使用基于树的模型进行提升计算，gblinear使用线性模型进行提升计算
        'objective': 'reg:linear',     # 定义学习任务及相应的学习目标，可选的目标函数如下：“reg:linear” —— 线性回归,“reg:logistic”—— 逻辑回归。
        #'multi:softmax'
        'gamma': 0.03,                      # 最小损失减少，模型在默认情况下，对于一个节点的划分只有在其loss function 得到结果大于0的情况下才进行，而gamma 给定了所需的最低loss function的值
        'max_depth': 8,                     # 数的最大深度。缺省值为6，树的深度越大，则对数据的拟合程度越高（过拟合程度也越高）。即该参数也是控制过拟合
        'lambda': 4,                        # L2 正则的惩罚系数
        'subsample': 0.8,                   # 用于训练模型的子样本占整个样本集合的比例。如果设置为0.5则意味着XGBoost将随机的从整个样本集合中随机的抽取出50%的子样本建立树模型，这能够防止过拟合。
        'colsample_bytree': 0.8,            # 在建立树时对特征采样的比例。缺省值为1。
#         'eval_metric':'mape_ln',
        'min_child_weight': 8.5,            # 孩子节点中最小的样本权重和，在现行回归模型中，这个参数是指建立每个模型所需要的最小样本数。该成熟越大算法越conservative。即调大这个参数能够控制过拟合。
        'silent': 1,                        # 取0时表示打印出运行时信息，取1时表示以缄默方式运行，不打印运行时信息。缺省值为0。
        'eta': 0.03,                        # 为了防止过拟合，更新过程中用到的收缩步长。通常最后设置eta为0.01~0.2
        'seed': 1000}                       # 随机数的种子。缺省值为0。
    num_boost_round = 1000
    plst  = params_xgb.items()
    model_xgb = xgb.train(plst, xgb_train, num_boost_round,evals=[(xgb_train,'train'),(xgb_test,'test')],early_stopping_rounds=20,verbose_eval=True)
    return model_xgb